In [18]:
import pandas as pd
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,make_scorer,f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot
from xgboost import XGBClassifier
import seaborn as sn 
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV
import numpy as np




In [22]:
import multiprocessing as mp
print(mp.cpu_count())

8


Cleaning the data


In [5]:
train_data= pd.read_csv('C:/Users/benedict halim/Desktop/Y2S2/CS3244/Project/Train_dataset.csv',index_col=0)
train_data=train_data.drop(['date_x', 'date_actual_fail','serial_number','model'], axis=1)
target=train_data['failure_actual_fail']
X_train,X_test,y_train,y_test=train_test_split(train_data.drop(['failure_actual_fail'],axis='columns'),target,test_size=0.2,random_state=1)

In [ ]:
#currently the data is unbalanced
unique, count = np.unique(y_train, return_counts=True)
y_train_dict_value_count = { k:v for (k,v) in zip(unique, count)}
print(y_train_dict_value_count)

Balancing the data via oversampling
1. 1:4
2. 1:3
3. 1:2
4. 1:1


In [17]:


#balance to 1:1
sm = SMOTE(random_state=3244,sampling_strategy = 1.0)
x_train_over, y_train_over = sm.fit_resample(X_train, y_train)

#balance to 1:4
sm = SMOTE(random_state=3244,sampling_strategy = 0.25)
x_train_over1, y_train_over1 = sm.fit_resample(X_train, y_train)

#balance to 1:3 
sm = SMOTE(random_state=3244,sampling_strategy = (1/3))
x_train_over2, y_train_over2 = sm.fit_resample(X_train, y_train)

#balance to 1:2
sm = SMOTE(random_state=3244,sampling_strategy = 0.5)
x_train_over3, y_train_over3 = sm.fit_resample(X_train, y_train)




{0: 25041, 1: 649}
{0: 25041, 1: 6260}


Balancing the data via undersampling
1. 1:4
2. 1:3
3. 1:2
4. 1:1


In [21]:
#balance to 1:1
sm = RandomUnderSampler(random_state=3244,sampling_strategy = 1.0)
x_train_under, y_train_under = sm.fit_resample(X_train, y_train)

#balance to 1:4
sm = RandomUnderSampler(random_state=3244,sampling_strategy = 0.25)
x_train_under1, y_train_under1 = sm.fit_resample(X_train, y_train)

#balance to 1:3 
sm = RandomUnderSampler(random_state=3244,sampling_strategy = (1/3))
x_train_under2, y_train_under2 = sm.fit_resample(X_train, y_train)

#balance to 1:2
sm = RandomUnderSampler(random_state=3244,sampling_strategy = 0.5)
x_train_under3, y_train_under3 = sm.fit_resample(X_train, y_train)

unique, count = np.unique(y_train_under3, return_counts=True)
y_train_dict_value_count = { k:v for (k,v) in zip(unique, count)}
print(y_train_dict_value_count)

{0: 1298, 1: 649}


Random Forest 


In [7]:

max_features_range = np.arange(1,6,1)
n_estimators_range = np.arange(10,210,10)
param_grid = dict(max_features=max_features_range, n_estimators=n_estimators_range)

model = RandomForestClassifier()

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5,scoring='f1')


In [8]:
grid.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_features': array([1, 2, 3, 4, 5]),
                         'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200])},
             scoring='f1')

In [12]:
grid_results = pd.concat([pd.DataFrame(grid.cv_results_["params"]),pd.DataFrame(grid.cv_results_["mean_test_score"], columns=["F1score"])],axis=1)
max(grid_results['F1score'])

0.577990270187213

In [ ]:
""" y_predicted=model.predict(X_test)
cf_matrix = confusion_matrix(y_test, y_predicted)

import seaborn as sns

ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Seaborn Confusion Matrix with labels\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
pyplot.show() """
